# Phase 1: Qwen3-0.8B Fine-tuning on CodeAlpaca

Training Qwen3-Zero-Coder-Reasoning-0.8B on sahil2801/CodeAlpaca-20k dataset

**Expected Time:** 2-3 hours on T4 GPU

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate peft bitsandbytes trl

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Configuration
MODEL_PATH = "/kaggle/input/qwen3-08b-coder-reasoning"  # Your uploaded dataset (corrected name)
OUTPUT_DIR = "/kaggle/working/qwen3-08b-phase1-codealpaca"
DATASET_NAME = "sahil2801/CodeAlpaca-20k"

# Training hyperparameters
BATCH_SIZE = 4
GRADIENT_ACCUMULATION = 4  # Effective batch size = 16
LEARNING_RATE = 2e-4
NUM_EPOCHS = 3
MAX_SEQ_LENGTH = 2048

In [ ]:
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Load model with 4-bit quantization for memory efficiency
print("Loading model with 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)
print(f"Model loaded on device: {model.device}")

In [ ]:
# Configure LoRA for parameter-efficient fine-tuning
print("Setting up LoRA...")
lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Load CodeAlpaca dataset
print("Loading CodeAlpaca-20k dataset...")
dataset = load_dataset(DATASET_NAME, split="train")
print(f"Dataset size: {len(dataset)} examples")
print(f"\nSample entry:")
print(dataset[0])

In [ ]:
# Format dataset for instruction tuning
def format_instruction(example):
    """Format Alpaca examples into instruction-following format"""
    instruction = example.get('instruction', '')
    input_text = example.get('input', '')
    output = example.get('output', '')
    
    if input_text:
        prompt = f"Instruction: {instruction}\n\nInput: {input_text}\n\nResponse:"
    else:
        prompt = f"Instruction: {instruction}\n\nResponse:"
    
    text = f"{prompt} {output}"
    return {"text": text}

dataset = dataset.map(format_instruction)
print("\nFormatted example:")
print(dataset[0]['text'][:500])

In [ ]:
# Training arguments (using SFTConfig for SFT-specific parameters)
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    fp16=True,
    save_strategy="epoch",
    logging_steps=50,
    warmup_steps=100,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    report_to="none",
    max_grad_norm=0.3,
    # SFT-specific parameters
    max_length=2048,
    dataset_text_field="text",
    packing=False,
)

In [ ]:
# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    processing_class=tokenizer,
    args=training_args,
)

print("\nTrainer initialized. Starting training...")


In [ ]:
# Train!
trainer.train()

In [ ]:
# Save the fine-tuned model
print("Saving model...")
trainer.model.save_pretrained(OUTPUT_DIR + "/final")
tokenizer.save_pretrained(OUTPUT_DIR + "/final")
print(f"✓ Model saved to {OUTPUT_DIR}/final")

In [ ]:
# Merge LoRA adapter into base model for Phase 2
print("\n" + "="*60)
print("MERGING LORA ADAPTER INTO BASE MODEL")
print("="*60)

# Reload the trained model in full precision for merging
print("\nLoading trained model for merging...")
from peft import PeftModel

# Load base model (full precision)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Load the LoRA adapter we just trained
lora_model = PeftModel.from_pretrained(base_model, OUTPUT_DIR + "/final")

# Merge LoRA weights into base model
print("Merging LoRA adapter into base model...")
merged_model = lora_model.merge_and_unload()

# Save the merged model
MERGED_OUTPUT = OUTPUT_DIR + "/merged"
print(f"Saving merged model to {MERGED_OUTPUT}...")
merged_model.save_pretrained(MERGED_OUTPUT)
tokenizer.save_pretrained(MERGED_OUTPUT)

print(f"\n✓ Merged model saved to {MERGED_OUTPUT}")
print("✓ This merged model should be used as input for Phase 2")
print(f"✓ Pad token ID: {tokenizer.pad_token_id} (preserved for next phase)")

# Clean up to free memory
del base_model, lora_model, merged_model
torch.cuda.empty_cache()
print("✓ Memory cleared")

## Merge LoRA Adapter into Base Model

For sequential training (Phase 1 → Phase 2 → Phase 3 → Phase 4), we need to merge the LoRA adapter into the base model so Phase 2 can build on this knowledge.

In [ ]:
# Test the model
print("\nTesting the fine-tuned model...")
test_prompt = "Instruction: Write a Python function to calculate factorial.\n\nResponse:"

inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.7)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

In [ ]:
# Archive BOTH outputs for download
print("\nArchiving outputs...")
!zip -r qwen3-08b-phase1-codealpaca-lora.zip {OUTPUT_DIR}/final
!zip -r qwen3-08b-phase1-codealpaca-merged.zip {OUTPUT_DIR}/merged

print("\n" + "="*60)
print("PHASE 1 COMPLETE!")
print("="*60)
print("\n✓ LoRA adapter archived: qwen3-08b-phase1-codealpaca-lora.zip")
print("✓ Merged model archived: qwen3-08b-phase1-codealpaca-merged.zip")
print("\n⚠️  IMPORTANT: Upload the MERGED model as input for Phase 2")
print(f"⚠️  Pad token ID {tokenizer.pad_token_id} is preserved in merged model")